In [1]:
#Get location of fixed and movimg image
#Customize for your own data
import os 
notebook_path = os.getcwd()
samplesPath = os.path.join(notebook_path, '../samples/')
print(samplesPath)
fixed_PATH = os.path.join(samplesPath, 'fixed.nii.gz')
assert os.path.exists(fixed_PATH), f'No file at {fixed_PATH}'
print(f'fixed_PATH {fixed_PATH}')
moving_PATH = os.path.join(samplesPath, 'moving.nii.gz')
assert os.path.exists(moving_PATH), f'No file at {moving_PATH}'
print(f'moving_PATH {moving_PATH}')

/mnt/data/supratik/deeds-registration/deeds/../samples/
fixed_PATH /mnt/data/supratik/deeds-registration/deeds/../samples/fixed.nii.gz
moving_PATH /mnt/data/supratik/deeds-registration/deeds/../samples/moving.nii.gz


In [2]:
#Install : pip install git+https://github.com/supratikbose/deeds-registration
from deeds import registration
import SimpleITK as sitk
#Volumes are expected to be have identical dimension and identical isometric pixel spacing 
fixed_vol_np = sitk.GetArrayFromImage(sitk.ReadImage(fixed_PATH))
moving_vol_np = sitk.GetArrayFromImage(sitk.ReadImage(moving_PATH))
#Invoke Deeds
moved_vol_np, flow_3channelUVW_np, flow_flattened_out_np, defVecShape =\
    registration(moving_vol_np, fixed_vol_np, defVectorResampledToVolume_in=True,alpha=1.6, levels=5, verbose=True)

#Interpreting input and result
#In the input and result volumes, i.e., in  moving_vol_np, fixed_vol_np and in 
#moved_vol_np we assume the 1st dimension is depth(Z of size o) followed by 
#height(Y of size n) and finally width(X of size m). Deformation vector is 
# also given out in 3-channel flow_3channelUVW_np where channels are of the 
# order [U,V,W]

#Examining deeds/libs/dataCostD.h/warpAffine() we interprete U,V,W as follows:
# During warping, fastest (innermost) loop is along W (size m); next 
# along H (size n) and finally along D (size o). Moreover
#U modifies x with x limit betwee 0 and N!! so U is displacement along 2nd dimension!! i.e., Height
# V modifies y with y limit betwee 0 and M!! so V is displacement along 3rd dimension!! i.e., Width
# W modifies z with z limit betwee 0 and O!! so W is displacement along 1st dimension!! i.e., Depth

#If defVectorResampledToVolume_in is false, it is of lower resolution than the input / output 
# volume. It is the same output whose flattened version is deformations.dat file.
# If defVectorResampledToVolume_in flag is true, flow_3channelUVW_np, and 
# flow_flattened_out_np are in full volume dimension.



Starting registration
Input shape: (256, 256, 256)
defVectorResampledToVolume_in: True
Output shape: (256,256,256)
Starting with identity transform.
MIND STEPS: 3 3 2 2 1
Level 0 grid=8 with sizes: 32x32x32 hw=8 quant=5
TMdDSTMdDS
Time: MIND=3.46351, data=4.14374, MST-reg=4.28533, transf.=4.02048
 speed=3.81986e+07 dof/s
std(J)=0.13 (J<0)=0e-7  SSD before registration: 0.00275497 and after 0.00275497

Level 1 grid=7 with sizes: 36x36x36 hw=7 quant=4
TMdDSTMdDS
Time: MIND=1.68224, data=4.25115, MST-reg=4.51739, transf.=4.07303
 speed=3.59157e+07 dof/s
std(J)=0.11 (J<0)=0e-7  SSD before registration: 0.00275497 and after 0.0015708

Level 2 grid=6 with sizes: 42x42x42 hw=6 quant=3
TMdDSTMdDS
Time: MIND=3.35499, data=2.33664, MST-reg=3.91651, transf.=4.07389
 speed=5.20606e+07 dof/s
std(J)=0.12 (J<0)=0e-7  SSD before registration: 0.00275497 and after 0.00141434

Level 3 grid=5 with sizes: 51x51x51 hw=5 quant=2
TMdDSTMdDS
Time: MIND=1.61941, data=2.03851, MST-reg=4.23107, transf.=4.0422
 s

In [3]:
#Helper function to deform volume using DVF returned by DEEDS

#vGiven deformation vector output flow_3channelUVW_np at full resolution (i.e. 
# defVectorResampledToVolume_in=True), one can use MONAI (1.1) to warp the 
#vmoving image to generate moved image outside Deeds as below:

def deformUsingDeedsDefVecAndMonaiWarp(vol_M_DHW, flow_3channelUVW_np):

    import torch
    from monai.networks.blocks import Warp as monai_warp
    a_monai_warp_transformer = monai_warp(mode='bilinear', padding_mode='border')
    
    vol_M_torch = torch.from_numpy(vol_M_DHW.astype('float32'))
    #Get u,v,w components from deeds_defVec
    u= torch.from_numpy(flow_3channelUVW_np[0,...].astype('float32'))
    v= torch.from_numpy(flow_3channelUVW_np[1,...].astype('float32'))
    w= torch.from_numpy(flow_3channelUVW_np[2,...].astype('float32'))

    #Add batch channel
    vol_M_torch_bc = vol_M_torch.unsqueeze(0).unsqueeze(0)
    #NOTE Since volume is in DHW, and from deeds we know w along D, u along H and v along w 
    #we need to pack deformation field in same DHW order. We also add batch and channel
    ddf_bc = torch.stack([w,u,v], dim=0).unsqueeze(0)

    #Apply Monai warp
    vol_MStarLocalDeeds_monai_torch = a_monai_warp_transformer(vol_M_torch_bc,  ddf_bc)
    return vol_MStarLocalDeeds_monai_torch.squeeze().detach().cpu().numpy().astype('float')

In [4]:
#Use  deformation vector returned by DEEDS to deform moving volume "locally" 
# and check mean square error (mse)

#Install : pip install sewar
from sewar.full_ref import mse

locally_moved_vol_np = deformUsingDeedsDefVecAndMonaiWarp(vol_M_DHW=moving_vol_np, flow_3channelUVW_np=flow_3channelUVW_np)

#MSE before deformation
mse_b4Def = mse(fixed_vol_np,  moving_vol_np)
print(f'mse before deformation {round(mse_b4Def,6)}') #mse_b4Def 0.002815
#MSE after deformation using DEEDS
mse_afterDef_Deeds = mse(fixed_vol_np,  moved_vol_np)
print(f'mse after deformation using Deeds {round(mse_afterDef_Deeds,6)}') #mse_afterDef_Deeds 0.000678
#MSE after local deformation using DVF returned by DEEDS
mse_afterLocalDef_Deeds_monai = mse(fixed_vol_np,  locally_moved_vol_np)
print(f'mse_afterLocalDef_Deeds_monai {round(mse_afterLocalDef_Deeds_monai,6)}') #mse_afterLocalDef_Deeds_monai 0.000678
#MSE between two deformed volumes (by DEEDS and by local deformation using DVF returned by DEEDS)
mse_between_two_methods = mse(moved_vol_np,  locally_moved_vol_np)
print(f' between two deformed volumes {round(mse_between_two_methods,6)}') #mse_between_two_methods 0.0

/opt/conda/envs/cloneMMIPython/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (None)/charset_normalizer (3.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/opt/conda/envs/cloneMMIPython/lib/python3.9/site-packages/monai/networks/blocks/warp.py:72: UserWarning: monai.networks.blocks.Warp: Using PyTorch native grid_sample.
  warnings.warn("monai.networks.blocks.Warp: Using PyTorch native grid_sample.")


mse before deformation 0.002815
mse after deformation using Deeds 0.000678


/opt/conda/envs/cloneMMIPython/lib/python3.9/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float32 and float64
  warnings.warn(msg)


mse_afterLocalDef_Deeds_monai 0.000678
 between two deformed volumes 0.0
